In [1]:
%run ../modules.ipynb
sys.path.append('../0. kernel-tutorials/')
from utilities.plotting import (
    plot_base, 
    plot_projection,
    plot_regression,
    plot_simple,
    get_cmaps,
    table_from_dict,
    check_mirrors,
)
plt.style.use("fivethirtyeight")
cmaps = get_cmaps()
# plt.style.use("../0. kernel-tutorials/utilities/kernel_pcovr.mplstyle")

dbl_fig = (2 * plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])


In [2]:
# si = element('Mg')
# (si)

In [3]:
"""The file has been created in zip format, reading it"""

frames_init_tot=ase.io.read('MP_complete_init.xyz', index=':')
frames_relax_tot=ase.io.read('MP_complete_relax.xyz', index=':')

In [4]:
select=5000
qb_red=random.sample(frames_init_tot, select)



In [5]:
"""description of nearest neighbor calculation here
https://github.com/materialsproject/pymatgen/blob/v2022.0.8/pymatgen/analysis/local_env.py#L3755-L4083"""

gr=['Alkali metals', 'Alkaline earth metals', 'Transition metals', 'Icosagens', 
             'Crystallogens', 'Pnictogens', 'Chalcogens', 'Halogens', 'Nobles gases']

frames_ase=[]

all_groups=[]
n_species=np.zeros(len(qb_red))
packing=np.zeros(len(qb_red))
# avg_coord=np.zeros(len(qb_red))
magic=np.zeros((len(qb_red)))
natoms=np.zeros((len(qb_red)))

nn_avg=np.zeros(len(qb_red))

nn_geo_avg=np.zeros(len(qb_red))

for j in range(len(qb_red)):
    print(j)
    frame=qb_red[j]
  
    frame.wrap(eps=1e-10)
    frames_ase.append(frame)
    frame_py=get_pymatgen(frame)
    natom=len(frame)
  
    
    if natom%4==0:
        magic[j]=1
    else:
        magic[j]=0
    natoms[j]=int(natom)
    
    v=frame.get_volume()  #unit = A^^3
    elements=frame.get_chemical_symbols()
   
    nn=np.zeros((len(elements)))
    nn_geo=np.zeros((len(elements))) 
    elements = np.array(elements)
    n_species[j]=(len(np.unique(elements)))
    V=0
    groups=np.zeros(len(gr))
    for el,i in zip(elements, range(len(elements))):
#         print(el)
        el_str=element(str(el))
        #empirical radius 
        r=el_str.atomic_radius
        #calculated radius, since the empirical one does not exist (??)
        if el=='Xe':
            r=float(108)
        if el == 'Kr':
            r=float(88)
        if el == 'Rn':
            r=float(120)
#         print(r)
        group_id=el_str.group_id

        group_type=group_conv(group_id)
        for a in range(len(gr)):
            if group_type == gr[a]:
                groups[a]+=1
        
        atom_vol=(4/3)*pi*(r/100)**3   #unit = A^^3
        V+=atom_vol
        nn[i]=CrystalNN(weighted_cn=False, distance_cutoffs=None,x_diff_weight=3.0,
                         search_cutoff=8, porous_adjustment=True).get_cn(frame_py, i)
       
        nn_geo[i]=CrystalNN(weighted_cn=False, distance_cutoffs=None, 
                        x_diff_weight=0.0, 
                         porous_adjustment=False).get_cn(frame_py, i)
        
#     avg_coord[j]=nn.mean()
    nn_avg[j]=nn.mean()
 
    nn_geo_avg[j]=nn_geo.mean()

    
    packing[j]=V/v
    all_groups.append(groups/natom) 
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
frames_ase=frames_ase[0:3056]
all_groups=all_groups[0:3056]
n_species=n_species[0:3056]
packing=packing[0:3056]

natoms=natoms[0:3056]

nn_avg=nn_avg[0:3056]
magic=magic[0:3056]
nn_geo_avg=nn_geo_avg[0:3056]



In [ ]:
# (all_groups)
dict={'frame':frames_ase, 'nn_avg':nn_avg, 
    'nn_geo_avg':nn_geo_avg,
    'natoms': natoms, 'packing_fraction':packing,'n_species': n_species, 
      'groups':all_groups, 'magic':magic} 

df= pd.DataFrame(dict)
df.head(10)

In [ ]:
plt.style.use("fivethirtyeight")
col=['#1f77b4', '#2ca02c', '#bcbd22','#ff7f0e',  '#d62728', '#9467bd', '#8c564b', '#e377c2','#7f7f7f']

m_gr_avg=(df['groups'].loc[df['magic'] == 1]).mean(axis=0)
nm_gr_avg=(df['groups'].loc[df['magic'] == 0]).mean(axis=0)

ex=[0,0,0.05,0,0,0,0,0,0]
labels=gr
fig, (ax_m, ax_n)=plt.subplots(1,2,figsize=(12,8))
patches, texts, auto =ax_m.pie(m_gr_avg,colors=col, autopct='%1.1f%%', 
                               explode=ex, pctdistance=1.17)
for auto in auto:
    auto.set_color('grey')
    
ax_m.axis('equal')
ax_m.set_title('Magic structures', color='dimgrey')
patches, texts, auto =ax_n.pie(nm_gr_avg, autopct='%1.1f%%', colors=col,
                               explode=ex, pctdistance=1.17)
# for text in texts:
#     text.set_color('grey')
for auto in auto:
    auto.set_color('grey')
ax_n.axis('equal')
ax_n.set_title('Rest of structures', color='dimgrey')
fig.legend(patches, labels, labelcolor='dimgrey', frameon=False, loc="lower center", fontsize=15, bbox_to_anchor=(0.5, -0.2))

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)
f, ax = plt.subplots(figsize=(9, 8))
sns.despine(left=True, bottom=True)
g=sns.lineplot(data=df, x='natoms', y='nn_avg', hue='magic', linestyle='--', legend=False)

i=sns.lineplot(data=df, x='natoms', y='nn_geo_avg', hue='magic', legend=False)
plt.xticks(c='grey')
plt.yticks(c='grey')
plt.ylabel('Average coordination number', c='grey')
plt.xlabel('Number of atoms in unit cell', c='grey')
plt.xlim(0,100)

plt.show()
plt.close()

In [ ]:
plt.style.use("fivethirtyeight")
f, ax = plt.subplots(figsize=(9,8))
plt.scatter(nn_avg, packing, c=magic, s=90, marker='+', label='chemical',
            cmap = colors.ListedColormap(['cornflowerblue', 'darkorange']), alpha=0.6)
plt.scatter(nn_geo_avg, packing, c=magic, s=90, label= 'structural',
            cmap = colors.ListedColormap(['cornflowerblue', 'darkorange']), alpha=0.6)
plt.xlabel('Average Coordination', color='dimgrey')
plt.ylabel('Packing Fraction', color='dimgrey')
plt.legend()

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)

f, ax = plt.subplots(figsize=(9, 8))
sns.despine(left=True, bottom=True)
g=sns.histplot(data=df, x='packing_fraction', hue='magic',stat='density', 
               common_norm=False,  multiple='dodge', shrink=.8, bins=20)
plt.xticks(c='grey')
plt.yticks(c='grey')
plt.ylabel('Count', c='grey')
plt.xlabel('Packing fraction (V_atoms/V_cell)', c='grey')
# plt.xlim(0,1)
plt.show()
plt.close()


f, ax = plt.subplots(figsize=(9, 8))
sns.despine(left=True, bottom=True)
g=sns.histplot(data=df, x='n_species', hue='magic',stat='density', 
               common_norm=False, multiple='dodge', shrink=.8, bins=int(max(n_species)))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.xticks(c='grey')
plt.yticks(c='grey')
plt.ylabel('Count', c='grey')
plt.xlabel('Number of species', c='grey')
plt.show()
plt.close()

In [ ]:
7plt.style.use("fivethirtyeight")
m=df.loc[df['magic'] == 1]
nm=df.loc[df['magic'] == 0]
print(m['packing_fraction'].mean(), nm['packing_fraction'].mean())
ex=[0.05,0,0]
# pie chart
labels = ['pf $\geq$ 0.9', '0.5 $\leq$ pf < 0.9', 'pf < 0.5']

sizes_m = [len(m.loc[(m['packing_fraction'] >= 0.7)]), 
         len(m.loc[(m['packing_fraction'] >= 0.5) & (m['packing_fraction'] < 0.7)]), 
         len(m.loc[(m['packing_fraction'] < 0.5)])]

sizes_nm = [len(nm.loc[(nm['packing_fraction'] >= 0.7)]), 
         len(nm.loc[(nm['packing_fraction'] >= 0.5) & (nm['packing_fraction'] < 0.7)]), 
         len(nm.loc[(nm['packing_fraction'] < 0.5)])]

fig, (ax_m, ax_n)=plt.subplots(1,2,figsize=(12,8))
patches, texts, auto =ax_m.pie(sizes_m, autopct='%1.1f%%',
                               explode=ex, pctdistance=1.18)

for auto in auto:
    auto.set_color('grey')
    
ax_m.axis('equal')
ax_m.set_title('Magic structures', color='dimgrey')
patches, texts, auto =ax_n.pie(sizes_nm, autopct='%1.1f%%',
                               explode=ex, pctdistance=1.18)
# for text in texts:
#     text.set_color('grey')
for auto in auto:
    auto.set_color('grey')

ax_n.axis('equal')
ax_n.set_title('Rest of structures', color='dimgrey')
fig.legend(patches, labels, labelcolor='dimgrey', frameon=False, loc="lower center", fontsize=15)

In [ ]:
plt.style.use("fivethirtyeight")
m=df.loc[df['magic'] == 1]
nm=df.loc[df['magic'] == 0]
print(m['packing_fraction'].mean(), nm['packing_fraction'].mean())
# ex=[0.05,0,0]
# pie chart
labels = ['FCC or HCP', 'BCC']

sizes_m = [len(m.loc[(m['packing_fraction'] ==0.74)]), 
         len(m.loc[(m['packing_fraction'] ==0.68)])]
sizes_nm = [len(nm.loc[(nm['packing_fraction'] ==0.74)]), 
         len(nm.loc[(nm['packing_fraction'] ==0.68)])]

fig, (ax_m, ax_n)=plt.subplots(1,2,figsize=(12,8))
patches, texts, auto =ax_m.pie(sizes_m, autopct='%1.1f%%',
                               pctdistance=1.18)

for auto in auto:
    auto.set_color('grey')
    
ax_m.axis('equal')
ax_m.set_title('Magic structures', color='dimgrey')
patches, texts, auto =ax_n.pie(sizes_nm, autopct='%1.1f%%',
                               explode=ex, pctdistance=1.18)
# for text in texts:
#     text.set_color('grey')
for auto in auto:
    auto.set_color('grey')

ax_n.axis('equal')
ax_n.set_title('Rest of structures', color='dimgrey')
fig.legend(patches, labels, labelcolor='dimgrey', frameon=False, loc="lower center", fontsize=15)

In [ ]:
# frames_ase=[]
# iso_q=[]
# natoms=[]


# for [structure_data] in qb_red[::10]:
#     atoms=structure_data.get_ase()
#     atoms.wrap()
#     pos=atoms.get_positions()
#     try:
#         vor=Voronoi(pos)
#     except:
#         continue
#     frames_ase.append(atoms)
#     natom = len(structure_data.get_site_kindnames())
#     natoms.append(natom)
#     vol=atoms.get_volume()
#     vor=Voronoi(pos)  
#     polygons = {}
#     for id, region_index in enumerate(vor.point_region):
#         points = []
#         for vertex_index in vor.regions[region_index]:
#             if vertex_index != -1:  # the library uses this for infinity
#                 points.append(list(vor.vertices[vertex_index]))
#         points.append(points[0])    
#         polygons[id]=points 
#     a_tot = sum(poly_area(polygons[i]) for i in range(len(polygons)))
#     iso=36*pi*vol**2/(a_tot**3)
#     iso_q.append(iso)
# len(iso_q)"""Removing Nan valies and infinity"""
# for l in range(len(lista)) if np.isinf(lista[l])!=True or np.isnan(lista[l])!=True]



# def remove_zeros(lista):
#     idx=[l for l in range(len(lista)) if np.isinf(lista[l])!=True and np.isnan(lista[l])!=True]
#     return idx

# indexes=remove_zeros(iso_q)
# IQ= list(itemgetter(*indexes)(iso_q))
# natoms_red=list(itemgetter(*indexes)(natoms))
# frames_ase=list(itemgetter(*indexes)(frames_ase))